In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [2]:
# use watermark to keep track of package versions
%load_ext watermark
%watermark -v -p pandas,numpy,matplotlib,torch

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.5.0

pandas    : 1.5.0
numpy     : 1.23.4
matplotlib: 3.6.1
torch     : 1.12.1+cu116



In [3]:
# load data
df = pd.read_csv("data_banknote_authentication.txt", header=None)
df.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [4]:
# split data into features and labels
X_features = df[[0,1,2,3]].values
y_labels = df[4].values

In [5]:
# check shape of features and labels
print(X_features.shape)

(1372, 4)


In [6]:
# check label distribution
np.bincount(y_labels)

array([762, 610])

### Define the dataloader

In [7]:
# define dataset class
class MyDataset(Dataset):
    def __init__(self, X, y):

        # define features and labels
        self.features = torch.tensor(X, dtype = torch.float32)
        self.labels = torch.tensor(y, dtype = torch.float32)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return x, y
    
    def __len__(self):
        return self.labels.shape[0]

In [8]:
# train size
train_size = int(0.8 * len(df))
train_size

1097

In [9]:
# validation size
val_size = X_features.shape[0] - train_size
val_size

275

In [10]:
# use torch.utils to generate dataloader
dataset = MyDataset(X_features, y_labels)

# split dataset into train and validation
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

In [11]:
# make train dataloader
train_loader = DataLoader(dataset = train_set,
                          batch_size = 10,
                          shuffle = True,
                          )

# make validation dataloader
val_loader = DataLoader(dataset = val_set,
                        batch_size = 10,
                        shuffle = False,)

In [12]:
# implement model
class LogisticRegression(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)

    def forward(self, x):
        logits = self.linear(x)
        preds = torch.sigmoid(logits)
        return preds

In [13]:
# model agnostic code
device = "cuda" if torch.cuda.is_available() else 'cpu'

In [14]:
# define model
model = LogisticRegression(num_features = 4).to(device)

# define optimizer
optimizer = optim.SGD(params = model.parameters(), lr = 0.1)

In [15]:
# define training loop
epochs = 10

for epoch in range(epochs):
    model.train()

    for batch_idx, (features, class_labels) in enumerate(train_loader):
        # make predictions
        preds = model(features.to(device))

        # calculate loss
        loss = F.binary_cross_entropy(preds, class_labels.view(preds.shape).to(device))

        # zero gradients
        optimizer.zero_grad()

        # backprop
        loss.backward()

        # optimizer step (gradient descent)
        optimizer.step()

        ### LOGGING
        if not batch_idx % 20: # every 20 batches
            print(f"Epoch: {epoch+1:03d}/{epochs:03d} | "
                  f"Batch {batch_idx:03d}/{len(train_loader):03d} | "
                  f"Loss: {loss:.2f}")

Epoch: 001/010 | Batch 000/110 | Loss: 1.80
Epoch: 001/010 | Batch 020/110 | Loss: 0.21
Epoch: 001/010 | Batch 040/110 | Loss: 0.26
Epoch: 001/010 | Batch 060/110 | Loss: 0.09
Epoch: 001/010 | Batch 080/110 | Loss: 0.04
Epoch: 001/010 | Batch 100/110 | Loss: 0.05
Epoch: 002/010 | Batch 000/110 | Loss: 0.04
Epoch: 002/010 | Batch 020/110 | Loss: 0.21
Epoch: 002/010 | Batch 040/110 | Loss: 0.09
Epoch: 002/010 | Batch 060/110 | Loss: 0.14
Epoch: 002/010 | Batch 080/110 | Loss: 0.04
Epoch: 002/010 | Batch 100/110 | Loss: 0.07
Epoch: 003/010 | Batch 000/110 | Loss: 0.05
Epoch: 003/010 | Batch 020/110 | Loss: 0.03
Epoch: 003/010 | Batch 040/110 | Loss: 0.07
Epoch: 003/010 | Batch 060/110 | Loss: 0.17
Epoch: 003/010 | Batch 080/110 | Loss: 0.07
Epoch: 003/010 | Batch 100/110 | Loss: 0.06
Epoch: 004/010 | Batch 000/110 | Loss: 0.14
Epoch: 004/010 | Batch 020/110 | Loss: 0.17
Epoch: 004/010 | Batch 040/110 | Loss: 0.04
Epoch: 004/010 | Batch 060/110 | Loss: 0.01
Epoch: 004/010 | Batch 080/110 |

### Check Accuaracy

In [21]:
# create function to compute accuracy
def compute_accuracy(model, dataloader):
    model.eval()

    correct = 0.0
    num_examples = 0

    for idx, (features, class_labels) in enumerate(dataloader):
        # make predictions
        with torch.inference_mode():
            pred = model(features.to(device))

        # calculate accuracy
        preds = torch.where(pred > 0.5, 1, 0)
        lab = class_labels.view(preds.shape).to(preds.dtype).to(device)
        compare = lab == preds
        correct += torch.sum(compare)
        num_examples += len(compare)
    
    return correct / num_examples

In [24]:
# compute accuracy on train and validation
train_acc = compute_accuracy(model, train_loader)
print(f"Train Accuracy: {train_acc*100:.2f}%")

train_val = compute_accuracy(model, val_loader)
print(f"Val Accuray {train_val * 100:2f}%")

Train Accuracy: 98.91%
Val Accuray 99.272728%
